# Résolution de système linéaire par la méthode du pivot de Gauss

## Les étapes du solveur:
* Lecture du fichier en entrée
* Création de la matrice augmentée stockant le système linéaire
* Sélection du pivot (maximum de la diagonale)
* Propagation du pivot sur les lignes concernées
* Résolution du problème linéaire
* Ecriture du résultat finale dans un fichier en sortie
* Génération de grandes matrices de tests

## 1. Lecture du fichier en entrée

La lecture du fichier ce fait à partir des outils de la librairie standard du C, grâce aux fonctions de **stdio.h** permettant la lecture d'un fichier. Cette lecture est ici faite élément par élément dans un fichier structuré de la façon suivante:
<br>
format du fichier d'entrée:
* n : nombre d'inconnues
* n lignes d'équations (n + 1 élements par lignes, séparation par un espace)

exemple: <br>
2<br>
2 1 5<br>
4 -6 -20<br>

## 2. Création de la matrice augmentée stockant le système linaire

Nous stockons le système linaire, lu à partir du fichier d'entrée, au sein de la structure suivante:
<br>
```C
/// @brief linear system wrapper structure which contains the system matrix as a 1D array (data) 
///        and a pointer array (storage) to facilitate access to elements inside the linear system matrix
typedef struct linear_system_t {
    int nb_unknowns;
    double * data;
    double ** storage;
} linear_system_t;
```

Cette façon de stocker notre système linéaire nous permet dans un permier temps d'acceder aux dimenssions de la matrice augmentée, à partir du premier membre de la structure **nb_unknowns**. De plus, nous pouvons stocker dans cette même structure à l'intérieur d'un tableau unidimensionnel **data** dans lequel tous les coefficients seront stockées séquentiellement. L'ordonancement de nos indices sous forme matricielle au sein du tableau 1D **data** ce fait à partir du tableau de pointeur **storage** qui nous permet d'acceder au élément de **data** comme un tableau bidimensionnel. Les avantages de ce stockage nous permet d'éviter de faire trop d'appel d'allocation dynamique de mémoire par **malloc**, et par conséquent de faire une économisation mémoire.

## 3. Sélection du pivot (maximum de la diagonale)

La stratégie de sélection du pivot est une étape cruciale dans la résolution par méthode de Gauss. Nous utilisons une méthode de pivot partiel, dans laquel nous sélectionnons la valeur la plus grande sur la diagonale. L'algorithme de propagation, vu plus tard, itère sur les lignes de notre système linéaire de la première à la dernière ligne. Il nous faut alors selectionner un nouveau pivot pour chaque ligne parcourue.<br>
Cette sélection de pivot sur la chaque ligne est décrite dans l'algorithme suivant:

```C
/// @brief Finds the pivot used in the currently provided linear system using a partial pivot strategy
/// @param linear_system the linear system
/// @param current_line
/// @param pivot_line
/// @return the pivot
double select_current_pivot(linear_system_t *linear_system, int current_line, int *pivot_line)
{
    int y;

    int nb_matrix_rows = linear_system->nb_unknowns;

    double pivot = 0;
    double p = linear_system->storage[current_line][current_line]; // pivot in diagonal
    double abs_val;
    double col_val;
    double p_abs;
    for (y = current_line + 1; y < nb_matrix_rows; y++)            // check for max value in same column
    {
        col_val = linear_system->storage[y][current_line];
        abs_val = fabs(col_val); //check if the absolute value of the pivot coefficient to be selected
        p_abs = fabs(p);
        pivot = MAX(p_abs, abs_val);
        if(p_abs != pivot) *pivot_line = y; //if the current pivot is updated, we update the line the pivot is on
        p = (pivot == abs_val) ? col_val : p;
    }

    return p;
}
```

Décrivons plus en détail cette fonction C. Nous sélectionnons d'abord comme valeur pivot le coefficient positionner sur la diagonnale de la ligne traité. Il nous suffit ensuite de regarder les coefficients se trouvant sur la même colonne du pivot actuel, sous celui-ci. Nous effectuons une comparaison (fonction **MAX**) par valeur absolue du pivot actuelle avec les valeurs sous celui-ci, ceci afin de sélectionner à la fin la valeur comme pivot la plus grande possible afin d'assurer une certaine stabilité numérique. Dans le cas où le pivot actuelle à été mise à jour, nous enregistrons alors la ligne sur laquelle ce nouveau coefficient se positionne.

## 4. Propagation du pivot sur les lignes concernées

Comme mentionnée précédemment, l'algorithme de propagation du pivot boucle sur chacune des lignes du système linéaire itérativement. Ce processus se retrouve dans le bloc de suivant, définissant cette algorithme de propagation écrit en C:

```C
void linear_system_propagation(linear_system_t *linear_system)
{
    int curr_line;
    int nb_matrix_rows = linear_system->nb_unknowns;
    double pivot;
    int pivot_line;

    //loop iterativly over each row of the linear system matrix
    for (curr_line = 0; curr_line < nb_matrix_rows; curr_line++)
    {
        pivot_line = curr_line;
        
        // selection du pivot pour chaque ligne de la matrice augmentée du systeme lineaire
        pivot = select_current_pivot(linear_system, curr_line, &pivot_line);

        // changement de ligne pour que le pivot soit sur la diagonale
        if (pivot_line != curr_line)
        {
            swap_linear_system_rows(linear_system, curr_line, pivot_line);
        }

        // pivotage de la matrice
        apply_pivot(linear_system, curr_line);
    }
}
```

Dans cette fonction, nous itérons sur chacune des lignes de notre matrice dans laquelle on nous sélectionnons le pivot, et si on constate que l'on a un coefficient trouvée plus grand dans la colonne du pivot sélectionnée, nous pouvons alors échanger la ligne actuelle avec la ligne qui a un plus grand coefficient. Dès lors, nous pouvons alors effectuer le pivotage de la matrice.

## 5. Résolution du problème linéaire

Avant d'obtenir les solutions potentieles du problème linéaire, nous devons d'abord appliquer le pivot trouver durant les étapes précédentes et le propager sur les lignes sous la ligne du pivot. Cette application du pivot s'effectue à partir de la fonction suivante:
<br>
```C
/// @brief Application de la technique de pivot
/// @param linear_system le système linéaire
/// @param pivot_line la ligne du pivot choisi dans le système linéaire
void apply_pivot(linear_system_t *linear_system, int pivot_line)
{
    int i, j;

    int nb_matrix_rows = linear_system->nb_unknowns;

    double pivot = linear_system->storage[pivot_line][pivot_line];
    //propagation du pivot sur les lignes en dessous
    for (i = pivot_line + 1; i < nb_matrix_rows; i++) 
    {
        //propagation du pivot sur les coefficient de chaque lignes
        for (j = pivot_line + 1; j <= nb_matrix_rows; j++) 
        {
            // A[i][j] = A[i][j] - ( (A[i][pi] / A[pi][pi]) * A[pi][j] )
            linear_system->storage[i][j] = linear_system->storage[i][j] - ((linear_system->storage[i][pivot_line] / pivot) * linear_system->storage[pivot_line][j]);
        }
        linear_system->storage[i][pivot_line] = 0;
    }
}
```

À l'intérieur de l'algorithme, nous avons une double boucle (for) imbriquée parcourant chaque coefficient de la matrice en commençant à la ligne et colonne +1 de la ligne de pivot (le pivot se trouvant sur la diagonale). En outre, ces itérations visent à mettre à jour les coefficients des variables dans chaque ligne sous le pivot. 
<br>
Après avoir mis à jour tous les coefficients dans une ligne, la colonne où se trouve le pivot est remise à zéro dans cette ligne (sauf pour la ligne du pivot) pour éviter les calculs inutiles lors des itérations ultérieures.


---
Dorénavant, nous avons une fonction qui nous permet d'effectuer la triangulation de la matrice donnée en entrée avec un algorithme basique de compléxité O(n^3). La dernière étape étant de résoudre le système linéaire en lui-même afin d'obtenir les solutions de ce dernier.
Cette résolution du problème linéaire et calcul des solutions potentielles ce fait à partir de la fonction suivante:


```C
/// @brief Solves the linear system
/// @param linear_system the given linear system
/// @return the solutions to the linear system inside an array
double *solve_linear_system(linear_system_t *linear_system)
{
    int j = 0;
    double result = 0;

    int nb_matrix_rows = linear_system->nb_unknowns;
    int nb_matrix_cols = nb_matrix_rows + 1;

    // allocate memory for the solutions array, initialized with zeros
    double *solutions = (double *)calloc(linear_system->nb_unknowns, sizeof(double));
    if (solutions == NULL)
    {
        fprintf(stderr, "Out of memory!\n");
        exit(EXIT_FAILURE);
    }

    for (int i = nb_matrix_rows - 1; i >= 0; i--)
    { // commencer à la dernière ligne
        result = 0;
        for (j = i; j < nb_matrix_cols - 1; j++)
        {
            if (i != j)
            { // if the the two aren't the same then an initial solution has been found
                // result += A[i][j] * R[j]
                result += linear_system->storage[i][j] * solutions[j];
            }
        }
        // R[i]=(A[i][dim-1]-result)/A[i][i]
        solutions[i] = (linear_system->storage[i][nb_matrix_cols - 1] - result) / linear_system->storage[i][i];
    }
    return solutions;
}
```

Dans cet algorithme, nous calculons les solutions possibles pour chaque ligne en commençant par la dernière ligne du système linéaire, maintenant triangularisé après application du pivot. Nous itérons alors dans la deuxième boucle (for) pour indice j la partie triangulaire de la matrice pour trouver les valeurs constantes utilisées afin de calculer les autres solutions dans les lignes supérieures. Chaque valeur solution trouvée à la ligne i est stockée individuellement dans un tableau unidimensionnel **solutions**, à l'indice i.

## 6. Ecriture du résultat finale dans un fichier en sortie

L'écriture du fichier ce fait de la même façons que la lecture, à partir des outils de la librairie standard du C, grâce aux fonctions de **stdio.h** permettant la lecture d'un fichier. Cette écriture est ici faite élément par élément dans un fichier structuré de la façon suivante:
<br>
Format du fichier de sortie:
* n : nb d'inconnues
* n lignes triangulées (la partie utile de la matrice résultat, on ne stocke pas le zero)
* n valeurs (résultats des inconnus)

Exemple: <br>
2 <br>
4.000 -6.000 -20.000 <br> 
4.000 15.000 <br>
0.625 3.750 <br>

# Execution du script de résolution de système linaire

In [2]:
%%bash
module load gcc/10.2.0_spack2021_gcc-10.2.0-gdnn
module load 2018/valgrind/3.13.0
make clean && make
./main "10x10_linear_system.txt"
valgrind ./main "10x10_linear_system.txt"

Loading gcc/10.2.0_spack2021_gcc-10.2.0-gdnn
  Loading requirement: gmp/6.1.2_spack2021_gcc-10.2.0-z76c
    isl/0.21_spack2021_gcc-10.2.0-jjlz mpfr/4.0.2_spack2021_gcc-10.2.0-glhn
    mpc/1.1.0_spack2021_gcc-10.2.0-bzdh zstd/1.4.5_spack2021_gcc-10.2.0-ak7l
Delete objects, temporary files...
Done.
Create objects...
Create executables...
Done.
Starting ./main...
Matrice augmentée générée:
2.000      1.000      0.000      4.000      2.000      1.000      0.000      4.000      2.000      1.000      1.000      
-4.000     -2.000     3.000      -7.000     -4.000     -2.000     3.000      -7.000     -4.000     1.000      4.000      
4.000      1.000      -2.000     8.000      0.000      -3.000     -12.000    -1.000     8.000      7.000      5.000      
0.000      -3.000     -12.000    -1.000     1.000      0.000      4.000      2.000      1.000      0.000      7.000      
2.000      1.000      -2.000     8.000      0.000      -3.000     0.000      -3.000     -12.000    -1.000     5.000      


==2994== Memcheck, a memory error detector
==2994== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==2994== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==2994== Command: ./main 10x10_linear_system.txt
==2994== 
==2994== 
==2994== HEAP SUMMARY:
==2994==     in use at exit: 0 bytes in 0 blocks
==2994==   total heap usage: 5 allocs, 5 frees, 2,176 bytes allocated
==2994== 
==2994== All heap blocks were freed -- no leaks are possible
==2994== 
==2994== For counts of detected and suppressed errors, rerun with: -v
==2994== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)
